In [8]:
import pandas as pd

iris = pd.read_csv("iris.data", header=None, names=["sepal_length", "sepal_width", 
                                                    "petal_length", "petal_width", "label"])
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [255]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

X = iris.drop(['label'], axis=1)
y = iris.label

# encode label
y_data = pd.get_dummies(y)

In [256]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F

X = Variable(torch.from_numpy(X.values)).float()
y = Variable(torch.from_numpy(y_data.values)).float()

In [291]:
class Model(torch.nn.Module):
    """
    Input  layer  4
    Hidden layer1 256
    Hidden layer2 128
    Hidden layer3 10
    Output layer  3
    """
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = torch.nn.Linear(4, 8)
        self.layer2 = torch.nn.Linear(8, 12)
        self.layer3 = torch.nn.Linear(12, 8)
        self.layer4 = torch.nn.Linear(8, 3)
        
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        output1 = F.relu(self.layer1(x))
        output2 = F.relu(self.layer2(output1))
        output3 = F.relu(self.layer3(output2))
        y_pred  = (self.layer4(output3))
        return y_pred

In [292]:
model = Model()

In [306]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.7)

In [313]:
epochs = 10
for epoch in range(epochs):
    y_pred = model(X)
    
    loss = criterion(y_pred, y)
    print(f"Epoch: {epoch+1}\t Loss: {loss.data.item()}")
    
    # zero gradients, perform a backward pass, and update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 1	 Loss: 0.11264175176620483
Epoch: 2	 Loss: 0.11257757991552353
Epoch: 3	 Loss: 0.1125107854604721
Epoch: 4	 Loss: 0.11243827641010284
Epoch: 5	 Loss: 0.11235662549734116
Epoch: 6	 Loss: 0.11226997524499893
Epoch: 7	 Loss: 0.11217355728149414
Epoch: 8	 Loss: 0.11206645518541336
Epoch: 9	 Loss: 0.11195170134305954
Epoch: 10	 Loss: 0.11183293163776398


In [314]:
torch.argmax(y_pred, 1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
        2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

Accuracy

In [325]:
import numpy as np
sum(torch.argmax(y_pred, 1).numpy() == np.argmax(y_data.values, 1)) / y_data.shape[0]

0.6933333333333334